In [10]:
import pandas as pd
from os import getcwd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from UliPlot.XLSX import auto_adjust_xlsx_column_width

In [24]:
strain_query = '''
SELECT  [StrainGauge].[gid]
      ,[gpn]
      ,[ver] as 'PCB ver'
      ,[toolno]
      ,[station]
      ,[Position]
      ,[TestRecord].judge
      ,[TestRecord].stress
      ,[TestRecord].WQ211
      ,[memo]
      ,[org]
      ,[createdata]
  FROM [SMTvip].[dbo].[StrainGauge]

  Left JOIN [SMTvip].[dbo].[TestRecord] 
  on [StrainGauge].[gid] = [TestRecord].[gid]
  where gpn = '105-04263-20'
  ORDER by profdate DESC
'''

connection_string = "DRIVER={SQL Server};SERVER=T1-PE-SUPPORT;DATABASE=PR_WEB2;UID=pieng;PWD=Q2iT5cwHJW3FH"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
result = pd.read_sql(strain_query, engine)

In [25]:
result.judge = result.judge.map({'OK':1, 'NG':0})
result.sort_values('createdata', ascending=False, inplace=True)
result.head()

,gid,gpn,PCB ver,toolno,station,Position,judge,stress,WQ211,memo,org,createdata
0,14436,105-04263-20,V2,1,ATE,I101,1,238.50,350,ATE2.0(治具變更:轉板設計)治具編號1.2,T1,2022-09-09 09:58:00
2,14436,105-04263-20,V2,1,ATE,I301,1,141.35,150,ATE2.0(治具變更:轉板設計)治具編號1.2,T1,2022-09-09 09:58:00
3,14436,105-04263-20,V2,1,ATE,I405,1,146.73,300,ATE2.0(治具變更:轉板設計)治具編號1.2,T1,2022-09-09 09:58:00
4,14436,105-04263-20,V2,1,ATE,I601,1,205.17,450,ATE2.0(治具變更:轉板設計)治具編號1.2,T1,2022-09-09 09:58:00
5,14436,105-04263-20,V2,1,ATE,I604,1,31.28,300,ATE2.0(治具變更:轉板設計)治具編號1.2,T1,2022-09-09 09:58:00


In [26]:
droped = result.drop_duplicates(subset=['PCB ver','station','Position','memo'])

In [35]:
pivot = droped.groupby(['station','memo','PCB ver','createdata','Position'])['judge'].sum().unstack('Position').sort_index(level=['station','createdata'],ascending=False)

In [33]:
pivot.style.highlight_min(color='red', axis=None)